### Gradio를 활용 파일을 업로드해서 RAGAS를 질문 답변 Q&A csv를 생성

우선 Gradio를 활용해서 파일을 로딩한다.

In [1]:
import os
import gradio as gr
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader, TextLoader, CSVLoader

# 환경 변수 로드
load_dotenv()

last_uploaded_file = None

def load_file(file):
    global last_uploaded_file
    """파일을 업로드하면 파일 로더로 문서를 읽어들이는 함수"""
    if file is None:
        return "파일을 업로드해주세요."
    try:
        if isinstance(file, str):
            file_path = file
            file_name = os.path.basename(file_path)
        else:
            return "파일을 업로드해주세요."
        
        # 파일 타입에 따른 로더 선택
        if file_name.lower().endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif file_name.lower().endswith('.txt'):
            loader = TextLoader(file_path, encoding='utf-8')
        elif file_name.lower().endswith('.csv'):
            loader = CSVLoader(file_path)
        else:
            return f"❌ 지원하지 않는 파일 형식입니다: {file_name}"
        
        # 문서 로드
        docs = loader.load()
        last_uploaded_file = docs
        # 미리보기용으로 처음 1~2개만 표시
        preview = '\n---\n'.join(doc.page_content[:500] for doc in docs[:2])
        return f"✅ 파일 '{file_name}'이 성공적으로 로드되었습니다!\n\n미리보기:\n{preview}"
    except Exception as e:
        return f"❌ 파일 처리 중 오류가 발생했습니다: {str(e)}"

def create_interface():
    with gr.Blocks(title="파일 로더 데모") as demo:
        gr.Markdown("# 📄 파일 로더 데모 (챗봇 없음)")
        gr.Markdown("문서를 업로드하면 파일 로더로 읽어들여 미리보기를 보여줍니다.")
        file_input = gr.File(
            label="📄 문서 업로드",
            file_types=[".pdf", ".txt", ".csv"],
            type="filepath"
        )
        output = gr.Textbox(
            label="파일 로드 결과",
            interactive=False,
            lines=20
        )
        file_input.change(
            fn=load_file,
            inputs=[file_input],
            outputs=[output]
        )
    return demo

demo = create_interface()
demo.launch()

/Users/poowoohj/Documents/prj/homework-llm-skt/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


### 로딩된 파일을 RAGAS를 활용 Q&A CSV를 생성한다.

In [2]:
demo.close()
docs = last_uploaded_file

from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문장을 구분하여 분할 - 정규표현식 사용 (문장 구분자: 마침표, 느낌표, 물음표 다음에 공백이 오는 경우)
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",    # TikToken 인코더 이름
    separators=['\n\n', '\n', r'(?<=[.!?])\s+'],   # 구분자
    chunk_size=300,
    chunk_overlap=20,
    is_separator_regex=True,      # 구분자가 정규식인지 여부
    keep_separator=True,          # 구분자 유지 여부
)

chunks = text_splitter.split_documents(docs)

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# OpenAI Embeddings 모델을 로드
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Chroma 벡터 저장소 생성하기
vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,    
    collection_name="skb_settop_box_user_guide", 
    persist_directory="./chroma_db",
    collection_metadata = {'hnsw:space': 'cosine'}, # l2, ip, cosine 중에서 선택 
)

# 결과 확인
print(f"저장된 Document 개수: {len(vector_store.get()['ids'])}")

# LLM 설정
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

# LLM과 임베딩 모델 초기화
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini", temperature=0.2))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

#(3) Q&A Data 생성
from ragas.testset.persona import Persona

# 페르소나 정의 (다양한 관점에서 질문 생성)
personas = [
    Persona(
        name="elderly_customer", 
        role_description="SKB 세트탑 박스를 사용하는 고령 고객으로, 기술에 대한 이해도가 낮은 일반 사용자로 엉뚱한 질문을 많이 합니다. 한국어만을 사용합니다.",
    ),
    Persona(
        name="technician",    
        role_description="SKB 세트탑 박스를 설치하고 관리하는 기술자로, 세트탑 박스의 기능과 사용법에 대한 이해도가 높습니다. 한국어만을 사용합니다.",
    ),
]

from ragas.testset import TestsetGenerator

# TestsetGenerator 생성
generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings, persona_list=personas)

# 합성 데이터 생성
dataset = generator.generate_with_langchain_docs(chunks, testset_size=50)
# 데이터 저장
dataset.to_pandas().to_csv('./data/qa_dataset.csv', index=False)
dataset.to_pandas()

Closing server running on port: 7862
저장된 Document 개수: 472


Generating Samples: 100%|██████████| 51/51 [00:15<00:00,  3.21it/s]


,user_input,reference_contexts,reference,synthesizer_name
0,헌법에 따라 이 법의 목적은 무엇인가요?,"[고용노동부 (임금근로시간정책과 - 제63조 적용제외, 특례업종) 044-202-7...",이 법은 헌법에 따라 근로조건의 기준을 정함으로써 근로자의 기본적 생활을 보장하고 ...,single_hop_specifc_query_synthesizer
1,"근로자란 무엇이며, 그 정의에 포함되는 조건은 무엇입니까?",[1. “근로자”란 직업의 종류와 관계없이 임금을 목적으로 사업이나 사업장에 근로를...,근로자란 직업의 종류와 관계없이 임금을 목적으로 사업이나 사업장에 근로를 제공하는 ...,single_hop_specifc_query_synthesizer
2,제50조에서 정의하는 소정근로시간이란 무엇인가요?,[말한다.\n6. “평균임금”이란 이를 산정하여야 할 사유가 발생한 날 이전 3개월...,"제50조에서 정의하는 소정근로시간이란 제50조, 제69조 본문 또는 「산업안전보건법...",single_hop_specifc_query_synthesizer
3,제1항제6호에 따라 산출된 금액이 통상임금보다 적을 때 어떻게 하나요?,[9. “단시간근로자”란 1주 동안의 소정근로시간이 그 사업장에서 같은 종류의 업무...,제1항제6호에 따라 산출된 금액이 그 근로자의 통상임금보다 적으면 그 통상임금액을 ...,single_hop_specifc_query_synthesizer
4,제5조 에 뭐가 잇나요?,"[제5조(근로조건의 준수) 근로자와 사용자는 각자가 단체협약, 취업규칙과 근로계약을...","제5조(근로조건의 준수)는 근로자와 사용자가 각자가 단체협약, 취업규칙과 근로계약을...",single_hop_specifc_query_synthesizer
5,법제처에서 제공하는 국가법령정보센터의 근로기준법 중 폭행 금지 조항에 대해 설명해 ...,[법제처 ...,법제처에서 제공하는 국가법령정보센터의 근로기준법 제8조(폭행의 금지)는 사용자가 사...,single_hop_specifc_query_synthesizer
6,제11조는 어떤 사업장에 적용되나요?,"[를 집행하기 위하여 필요한 시간을 청구하면 거부하지 못한다. 다만, 그 권리 행사...",제11조에 따르면 이 법은 상시 5명 이상의 근로자를 사용하는 모든 사업 또는 사업...,single_hop_specifc_query_synthesizer
7,제2장 근로계약에 따라 사용자가 근로자에게 법령 주요 내용과 취업규칙을 어떻게 게시...,[출석하여야 한다. <개정 2010. 6. 4.>\n \n제14조(법령 주요 내용 ...,"제2장 근로계약에 따르면, 사용자는 이 법과 이 법에 따른 대통령령의 주요 내용과 ...",single_hop_specifc_query_synthesizer
8,"제16조에 따르면 근로계약의 계약기간은 어떻게 제한되며, 예외 사항에는 어떤 것들이...",[제2장 근로계약\n \n제15조(이 법을 위반한 근로계약) ① 이 법에서 정하는 ...,제16조에 따르면 근로계약은 기간을 정하지 아니한 것과 일정한 사업의 완료에 필요한...,single_hop_specifc_query_synthesizer
9,"대통령령이 근로기준법에서 임금의 구성항목, 계산방법, 지급방법 등의 변경에 어떤 영...",[법제처 ...,"대통령령으로 정하는 사유로 인해 단체협약 또는 취업규칙의 변경 등 임금의 구성항목,...",single_hop_specifc_query_synthesizer
